# PWV03 : Compare PWV with FGCM

**Goal :** PWV vs time in Spectrogram and Spectrum, and difference per observation

- author Sylvie Dagoret-Campagne
- creation date 2026-02-07 : version run2026_v01
- last update : 2026-02-07
- affiliation : IJCLab
- Kernel @usdf **w_2026_02*
- Home emac : base (conda)
- laptop : conda_py313

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
# must install the mysitcom package by doing at top level "pip install --user -e . "
from mysitcom.auxtel.pwv import scatter_datetime
from mysitcom.auxtel.pwv import strip_datetime
from mysitcom.auxtel.pwv import bar_counts_by_night
from mysitcom.auxtel.pwv import plot_dccd_chi2_vs_time
from mysitcom.auxtel.pwv import plot_dccd_chi2_vs_time_by_filter
from mysitcom.auxtel.pwv import stripplot_target_vs_time
from mysitcom.auxtel.pwv import plot_dccd_chi2_vs_time_by_target_filter
from mysitcom.auxtel.pwv import plot_dccd_chi2_histo_by_target_filter
from mysitcom.auxtel.pwv import plot_dccd_chi2_vs_time_by_target_filter_colorsedtype
from mysitcom.auxtel.pwv import plot_dccd_chi2_histo_by_target_filter_colorsedtype
from mysitcom.auxtel.pwv import summarize_dccd_chi2
from mysitcom.auxtel.pwv import plot_atmparam_vs_time, plot_atmparam_diff_vs_time
from mysitcom.auxtel.pwv import plot_atmparam_hist_per_filter, plot_atmparam_diff_hist_per_filter
from mysitcom.auxtel.pwv import plotcompare_atmparam_fgcm_vs_time

from mysitcom.auxtel.pwv import GetNightMidnightsDict,GetNightBoundariesDict

In [ ]:
import os

In [ ]:
# where are stored the figures
pathfigs = "figs_PWV03"
prefix = "pwv03"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

In [ ]:
import numpy as np
from numpy.linalg import inv
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor="white", alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy import constants as c

from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
from pandas.api.types import is_datetime64_any_dtype

import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (16,8)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams["legend.fontsize"] = "xx-large"

import scipy
from scipy.optimize import curve_fit,least_squares

from pprint import pprint

# new color correction model
import pickle
from scipy.interpolate import RegularGridInterpolator

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
# Remove to run faster the notebook
import ipywidgets as widgets
%matplotlib widget

In [ ]:
from PWV00_parameters import *

In [ ]:
DumpConfig()

In [ ]:
from importlib.metadata import version

In [ ]:
# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

In [ ]:
np.__version__

In [ ]:
pd.__version__

### Configuration

In [ ]:
def convertNumToDatestr(num):
    year = num//10_000
    month= (num-year*10_000)//100
    day = (num-year*10_000-month*100)

    year_str = str(year).zfill(4)
    month_str = str(month).zfill(2)
    day_str = str(day).zfill(2)
    
    datestr = f"{year_str}-{month_str}-{day_str}"
    return pd.to_datetime(datestr)

In [ ]:
PWVMIN = 0.
PWVMAX = 20.

In [ ]:
FLAG_WITHCOLLIMATOR = False
DATE_WITHCOLLIMATOR = 20230930
datetime_WITHCOLLIMATOR = convertNumToDatestr(DATE_WITHCOLLIMATOR)
datetime_WITHCOLLIMATOR = pd.to_datetime("2023-09-30 00:00:00.0+0000")
datetime_WITHCOLLIMATOR

## Initialisation

### Read the file
- `atmfilename` is defined in `PWV00_parameters.py` 

In [ ]:
the_suptitle = butlerusercollectiondict[version_run] 

In [ ]:
inputfilename = atmfilename.split("/")[-1]

if "parquet" in inputfilename:
    df_spec = pd.read_parquet(atmfilename)
elif "npy" in inputfilename:
    specdata = np.load(atmfilename,allow_pickle=True)
    df_spec = pd.DataFrame(specdata)
    df_spec["D_CCD [mm]"] = df_spec["D2CCD"]
    df_spec["PWV [mm]"] = df_spec["PWV [mm]_x"] 
    df_spec["PWV [mm]_rum"] = df_spec["PWV [mm]_y"] 
    df_spec["PWV [mm]_err"] = df_spec["PWV [mm]_err_x"] 
    df_spec["PWV [mm]_err_rum"] = df_spec["PWV [mm]_err_y"] 


    cols = [
    "PWV [mm]",
    "PWV [mm]_rum",
    "PWV [mm]_err",
    "PWV [mm]_err_rum",
    ]

    df_spec = df_spec.dropna(subset=cols)
    
else:
    raise "bad path of filename {inputfilename}"
    

In [ ]:
FLAG_RENAME_SPECTROGRAM_VARIABLES = True

if FLAG_RENAME_SPECTROGRAM_VARIABLES:
    df_spec.rename(
    {
    "chi2":"chi2_ram",
    "A1":"A1_ram",
    "A1_err": "A1_err_ram",
    "A2": "A2_ram",
    "A2_err": "A2_err_ram",
    "A3": "A3_ram",
    "A3_err": "A3_err_ram", 
    "VAOD": "VAOD_ram", 
    "VAOD_err": "VAOD_err_ram", 
    "angstrom_exp" : "angstrom_exp_ram", 
    "angstrom_exp_err" : "angstrom_exp_err_ram" , 
    "ozone [db]" :"ozone [db]_ram", 
    "ozone [db]_err": "ozone [db]_err_ram", 
    "PWV [mm]":  "PWV [mm]_ram",
    "PWV [mm]_err":"PWV [mm]_err_ram" , 
    "B": "B_ram" , 
    "B_err" : "B_err_ram", 
    "A_star": "A_star_ram" , 
    "A_star_err": "A_star_err_ram" , 
    "D_CCD [mm]" : "D_CCD [mm]_ram", 
    "D_CCD [mm]_err": "D_CCD [mm]_err_ram" 
    }
    ,axis=1,inplace = True)


In [ ]:
print(" | ".join(df_spec.columns)) 

In [ ]:
#df_spec.dtypes.to_frame('Type de donnée')

In [ ]:
# add time for plotting
#df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"])
df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"],utc=True)

In [ ]:
df_spec["nightObs"] = df_spec.apply(lambda x: x['id']//100_000, axis=1)

In [ ]:
df_spec["seq_num"]  = df_spec["id"] % 100_000

## Select only empty and OG550 filters

In [ ]:
df_spec["FILTER"].unique()

In [ ]:
if FLAG_PWVFILTERS: 
    df_spec = df_spec[df_spec["FILTER"].isin(PWV_FILTER_LIST) ]

### Check Filters

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 8))

strip_datetime(
    df=df_spec,
    x="Time",
    y="FILTER",
    hue="FILTER",
    ax=ax,
    size=9,
)

plt.show()


## Special study on Star Color (Spectral type)
- Load magnitudes from external file
- the magnitudes have been computed in another notebooks in ../2025-10-29-TOOLS



In [ ]:
targets_mag_files = "../2025-10-29-TOOLS/data/targets_magnitudes.csv"
df_targets_mag = pd.read_csv(targets_mag_files,index_col=0)      
df_targets_mag = df_targets_mag.sort_values(by="y")

### palette with SED type

In [ ]:
df_col = df_targets_mag.copy()
df_col = df_col.sort_values(by="B_V")

SpT = df_col["Sp_T"].values
unique_types = list(dict.fromkeys(SpT))  # garde l'ordre d'apparition
N_types = len(unique_types)


# Associe chaque type spectral à un entier
type_to_idx = {t: i for i, t in enumerate(unique_types)}
idx = np.array([type_to_idx[t] for t in SpT])

# Crée la colormap
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=-0.5, vmax=N_types - 0.5)

# Colorbar horizontale
fig, ax = plt.subplots(figsize=(14, 0.4), layout="constrained")
cbar = fig.colorbar(
    mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
    cax=ax,
    orientation="horizontal",
    ticks=range(N_types),
)
cbar.ax.set_xticklabels(unique_types, rotation=45, ha="right")
cbar.set_label("Spectral Type", fontsize=12,labelpad=10)

plt.show()

In [ ]:
df_targets_mag.head() 

### Target ordered by colors in the input file

In [ ]:
sorted_targets =  list(df_targets_mag.index) 
print(sorted_targets)

### Target ordered by colors in our data  file

In [ ]:
# order pf selected target by magnitude Y
order_selected_targets = [t for t in sorted_targets if t in df_spec["TARGET"].unique()]
print(order_selected_targets)

### build a palette for colors

In [ ]:
# --- Palette personnalisée cohérente avec la colormap jet ---
target_to_color = {}
for target in order_selected_targets:
    sp_type = df_targets_mag.loc[target, "Sp_T"]
    rgba = cmap(norm(type_to_idx[sp_type]))

    # Convert to pure Python floats
    target_to_color[target] = tuple(float(c) for c in rgba)

# Plot PWV vs time before application cuts

### Calculate difference and ratio for some params

In [ ]:
denom = np.sqrt(df_spec["PWV [mm]_err_ram"]**2 + df_spec["PWV [mm]_err_rum"]**2)

df_spec["diff_PWV_norm"] = np.where(
    np.isfinite(denom) & (denom > 0),
    (df_spec["PWV [mm]_ram"] - df_spec["PWV [mm]_rum"]) / denom,
    np.nan
)

df_spec["diff_PWV"] =  (df_spec["PWV [mm]_ram"] - df_spec["PWV [mm]_rum"]) 
df_spec["diff_PWV_err"] = np.sqrt( (df_spec["PWV [mm]_err_ram"]**2 - df_spec["PWV [mm]_err_rum"]**2)) 

## Calculate midnights and night boundaries

In [ ]:
DT = pd.Timedelta(minutes=7*24*60)
TMIN  = df_spec["Time"].min()-DT
TMAX  = df_spec["Time"].max()+DT

In [ ]:
# get night boundaries
dn = GetNightBoundariesDict(df_spec)
# get midnights
dnidnights = GetNightMidnightsDict(df_spec)

### Plot PWV in spectrogram vs Time before quality cuts

In [ ]:
fig,ax = plot_atmparam_vs_time(
    df_spec,
    time_col= "Time",
    filter_col = "FILTER",
    param_col = "PWV [mm]_ram",
    param_err_col = "PWV [mm]_err_ram",
    title_param = "PWV vs time (spectrogram no cut qual. cut)",
    
    # seuils / bornes
    param_min_fig=PWVMIN,
    param_max_fig=PWVMAX,
    param_min_cut=None,
    param_max_cut=None,
 
    # titres
    suptitle= the_suptitle,

    # axes externes
    axs=None,
    figsize=(18, 6),
)

if version_run not in ["run_v12"]:
    for key, tt in dn.items():
        ax.axvspan(tt[0],tt[1], color='blue', alpha=0.05,lw=0.5)

for key, midn in dnidnights.items():
    ax.axvline( midn ,color="purple",ls=":",alpha=0.5,lw=0.5)

### Plot PWV in spectrum vs Time

In [ ]:
fig,ax = plot_atmparam_vs_time(
    df_spec,
    time_col= "Time",
    filter_col = "FILTER",
    param_col = "PWV [mm]_rum",
    param_err_col = "PWV [mm]_err_rum",
    title_param = "PWV vs time (spectrum no cut qual. cut)",
    
    # seuils / bornes
    param_min_fig=PWVMIN,
    param_max_fig=PWVMAX,
    param_min_cut=None,
    param_max_cut=None,
 
    # titres
    suptitle= the_suptitle,

    # axes externes
    axs=None,
    figsize=(18, 6),
)

if version_run not in ["run_v12"]:
    for key, tt in dn.items():
        ax.axvspan(tt[0],tt[1], color='blue', alpha=0.05,lw=0.5)

for key, midn in dnidnights.items():
    ax.axvline( midn ,color="purple",ls=":",alpha=0.5,lw=0.5)

## Compare with 

In [ ]:
df_fgcm = pd.read_csv(fimename_fgcm) 

In [ ]:
df_fgcm["physicalFilter"].unique()

In [ ]:
df_fgcm["Time"] = pd.to_datetime(df_fgcm["Time"],utc=True)

In [ ]:
fig,ax = plotcompare_atmparam_fgcm_vs_time(
        df_spec,
        df_fgcm, 
        time_col= "Time",
        filter_col = "FILTER",
        param_col = "PWV [mm]_ram",
        time_fgcm_col = "Time",
        filter_fgcm_col = "physicalFilter",
        param_fgcm_col = "pwv",
        param_err_col = "PWV [mm]_err_rum",
        title_param = "PWV (Auxtel) and PWV (FGCM) vs time",
    
        # seuils / bornes
        param_min_fig=PWVMIN,
        param_max_fig=PWVMAX,
        param_min_cut=None,
        param_max_cut=None,
 
        # titres
        suptitle= the_suptitle,

        # axes externes
        axs=None,
        figsize=(18, 6),
    )


In [ ]:
if version_run not in ["run_v12"]:
    for key, tt in dn.items():
        ax.axvspan(tt[0],tt[1], color='blue', alpha=0.05,lw=0.5)

for key, midn in dnidnights.items():
    ax.axvline( midn ,color="purple",ls=":",alpha=0.5,lw=0.5)